In [157]:
import os
import json
import pandas as pd

from sqlalchemy import (
    Connection,
    CursorResult,
    Engine,
    Select,
    TextClause,
    create_engine,
    func,
    select,
)

from collections import defaultdict
from math import radians, cos, sin, asin, sqrt

from sqlalchemy.sql.expression import text

from datetime import datetime, timedelta
from numbers import Real
from typing import Any, Iterator, Tuple, Union

from numpy import positive

import geopandas as gpd
from shapely.geometry import Point, Polygon

In [5]:
timelike = Union[str, Real, datetime, pd.Timestamp]

def to_datetime(time: timelike) -> pd.Timestamp:
    if isinstance(time, str):
        time = pd.Timestamp(time, tz="utc")
    elif isinstance(time, datetime):
        time = pd.to_datetime(time, utc=True)
    elif isinstance(time, Real):
        time = pd.Timestamp(float(time), unit="s", tz="utc")
    return time

In [131]:
LAT_MIN, LAT_MAX = 50.896393, 50.967115
LON_MIN, LON_MAX = 6.919968, 7.005756
ALT_MIN, ALT_MAX = 0, 750 # update from 700 m to 750 m, in line with CTR limit at 2500 ft plus margin (and accounting for Geoid Height)

TIME_BETWEEN_TRAJS = 30

# SERA.5005(f)(1) criteria
ALERT_DISTANCE_M = 600      # alert distance wrt obstacles (should be 600)
ALERT_DELTA_HEIGHT_M = 300   # delta height (should be 300)

CPA_MARGIN_M = 20 # allowance for lateral distance to obstacle
DIP_MARGIN_M = 20 # allowance for dip below minimum height (45m corresponds to GVA = 2)
N_MIN = 5

GEOID_HEIGHT_M = 47  # geoid height for Cologne

#### 1 - Setup OpenSky Network Trino credentials

In [7]:
OSN_secrets_json = '/Users/patatino/Nextcloud/Documents/Survol/Trino_Access/trino_secrets.json'

with open(OSN_secrets_json) as OSN_secrets:
  OSN_creds = json.load(OSN_secrets)

os.environ['OPENSKY_USERNAME'] = OSN_creds['OPENSKY_USERNAME']
os.environ['OPENSKY_PASSWORD'] = OSN_creds['OPENSKY_PASSWORD']

#### 2 - Test connection with basic SQL query

In [9]:
from pyopensky.trino import Trino

trino = Trino()

query = "select * from state_vectors_data4 limit 5"

df = trino.query(query, cached = False)

FINISHED: : 31.8% [00:00, 148%/s] 
DOWNLOAD: 5.00lines [00:00, 562lines/s]


In [10]:
df.columns

Index(['time', 'icao24', 'lat', 'lon', 'velocity', 'heading', 'vertrate',
       'callsign', 'onground', 'alert', 'spi', 'squawk', 'baroaltitude',
       'geoaltitude', 'lastposupdate', 'lastcontact', 'serials', 'hour'],
      dtype='object')

# 3 - Send queries and merge results

In [71]:
start_str = "01/07/24"
end_str = "05/07/24"

start = datetime.strptime(start_str, '%d/%m/%y')
end = datetime.strptime(end_str, '%d/%m/%y')

# Modify start_date to be 00:00:00
start = start.replace(hour=0, minute=0, second=1, microsecond=0)
# Modify end_date to be 23:59:59
end = end.replace(hour=23, minute=59, second=59, microsecond=999999)

start_time = int(start.timestamp())
start_hour = start_time - (start_time % 3600)
end_time = int(end.timestamp())
end_hour = end_time - (end_time % 3600)

In [73]:
svdata4_query = (
        f"SELECT * FROM state_vectors_data4"
        f" WHERE icao24 LIKE '%'"
        f" AND time >= {start_time} AND time <= {end_time}"
        f" AND hour >= {start_hour} AND hour <= {end_hour}"
        f" AND lat >= {LAT_MIN} AND lat <= {LAT_MAX}"
        f" AND lon>= {LON_MIN} AND lon <= {LON_MAX}"
        f" AND geoaltitude >= {ALT_MIN} AND geoaltitude <= {ALT_MAX}"
        f" ORDER BY time"
    )

trino = Trino()

svdata4_df = trino.query(
    svdata4_query,
    cached=False,
    compress=True,
)

FINISHED: : 98.0% [00:00, 925%/s]
DOWNLOAD: 3.98klines [00:00, 424klines/s]


In [74]:
icao_list = svdata4_df.icao24.unique()
icao24_str = ', '.join(f"'{item}'" for item in icao_list)

ops_sts_query = (
    f"SELECT icao24, mintime, maxtime, nacv, systemdesignassurance, version, positionnac, geometricverticalaccuracy, sourceintegritylevel, barometricaltitudeintegritycode  FROM operational_status_data4"
    f" WHERE icao24 IN ({icao24_str})"
    f" AND mintime >= {start_time} AND maxtime <= {end_time}"
    f" AND hour >= {start_hour} AND hour <= {end_hour}"
    f" ORDER by mintime"
)

print('Connecting to OSN database...')
trino = Trino()
ops_sts_df = trino.query(
    ops_sts_query,
    cached=False,
)

ops_sts_df['time'] = ops_sts_df['mintime'].astype('int64')

Connecting to OSN database...


FINISHING: : 100% [00:00, 139%/s] 
DOWNLOAD: 153klines [00:01, 125klines/s]


In [75]:
# Initialize an empty DataFrame to hold the results
merged_df = pd.DataFrame()

# Loop over each unique 'icao24' in both dataframes
unique_icao24s = pd.concat([svdata4_df['icao24'], ops_sts_df['icao24']]).unique()

for icao24 in unique_icao24s:
    # Filter each dataframe by 'icao24'
    sub_df1 = svdata4_df[svdata4_df['icao24'] == icao24]
    sub_df2 = ops_sts_df[ops_sts_df['icao24'] == icao24]

    # Ensure both sub-dataframes are sorted by 'time'
    sub_df1 = sub_df1.sort_values('time')
    sub_df2 = sub_df2.sort_values('time')

    # Perform merge_asof on the filtered and sorted dataframes
    merged_sub_df = pd.merge_asof(sub_df1, sub_df2, on='time', by='icao24', direction='backward')
    
    # Append the result to the main dataframe
    merged_df = pd.concat([merged_df, merged_sub_df], ignore_index=True)


In [76]:
posdata4_query = (
    f"SELECT mintime, icao24, nic  FROM position_data4"
    f" WHERE icao24 IN ({icao24_str})"
    f" AND lat >= {LAT_MIN} AND lat <= {LAT_MAX}"
    f" AND lon>= {LON_MIN} AND lon <= {LON_MAX}"
    f" AND mintime >= {start_time} AND maxtime <= {end_time}"
    f" AND hour >= {start_hour} AND hour <= {end_hour}"
    f" ORDER by mintime"
)

print('Connecting to OSN database...')
trino = Trino()
posdata4_df = trino.query(
    posdata4_query,
    cached=False,
)


Connecting to OSN database...


FINISHED: : 100% [00:00, 945%/s]
DOWNLOAD: 9.83klines [00:00, 1.35Mlines/s]


In [77]:
posdata4_df['time'] = posdata4_df['mintime'].astype('int64')

In [135]:
# Initialize an empty DataFrame to hold the results
final_df = pd.DataFrame()

for icao24 in unique_icao24s:
    # Filter each dataframe by 'icao24'
    sub_df1 = merged_df[merged_df['icao24'] == icao24]
    sub_df2 = posdata4_df[posdata4_df['icao24'] == icao24]

    # Ensure both sub-dataframes are sorted by 'time'
    sub_df1 = sub_df1.sort_values('time')
    sub_df2 = sub_df2.sort_values('time')

    # Perform merge_asof on the filtered and sorted dataframes
    merged_sub_df = pd.merge_asof(sub_df1, sub_df2, on='time', by='icao24', direction='backward')
    
    # Append the result to the main dataframe
    final_df = pd.concat([final_df, merged_sub_df], ignore_index=True)

final_df = final_df.drop(columns=['hour', 'mintime_x', 'maxtime', 'mintime_y'])


In [136]:
final_df.columns

Index(['time', 'icao24', 'lat', 'lon', 'velocity', 'heading', 'vertrate',
       'callsign', 'onground', 'alert', 'spi', 'squawk', 'baroaltitude',
       'geoaltitude', 'lastposupdate', 'lastcontact', 'serials', 'nacv',
       'systemdesignassurance', 'version', 'positionnac',
       'geometricverticalaccuracy', 'sourceintegritylevel',
       'barometricaltitudeintegritycode', 'nic'],
      dtype='object')

# 4 - Add DEM ground elevation information

In [93]:
import rasterio
from pyproj import Transformer, transform
import numpy as np

In [96]:
crs_transformer = Transformer.from_crs(4326, 3035, always_xy = True) # Transformer from WGS-84 to ETRS89-LAEA

def transform_coords(lon, lat):
    return crs_transformer.transform(lon, lat)

In [137]:
final_df['gnd_elev'] = np.nan

dem_src = rasterio.open('./resources/Cologne_EUDEM_v11.tif')

final_df['etrs89_x'], final_df['etrs89_y'] = zip(*final_df.apply(lambda row: transform_coords(row['lon'], row['lat']), axis=1))


In [138]:
def get_elevation(x, y, dem):
    row, col = dem.index(x, y)
    return dem.read(1)[row, col]

final_df['gnd_elev'] = final_df.apply(lambda row: get_elevation(row['etrs89_x'], row['etrs89_y'], dem_src), axis=1)

In [139]:
final_df = final_df.drop(columns=['etrs89_x', 'etrs89_y'])

# Process df with distance information

In [140]:
def haversine(pt1, pt2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    Returned units are in metres. Differs slightly from PostGIS geography
    distance, which uses a spheroid, rather than a sphere.
    """

    lat1, lon1 = pt1[0], pt1[1]
    lat2, lon2 = pt2[0], pt2[1]

    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2.0) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2.0) ** 2
    c = 2 * asin(sqrt(a))
    r = 6371000  # Radius of earth in m
    return c * r

In [141]:
final_df["prev_time"] = final_df.time.shift()
final_df['closest_obst_name'] = 'ground'
final_df['inf_flt'] = False
final_df['inf_pt'] = False
final_df['gnd_inf_flt'] = False
final_df['gnd_inf_pt'] = False
final_df['min_hgt'] = final_df['gnd_elev'] + 300 # Minimum height away from obstacles is 300 m above ground (over congested areas)

map_time_traj = defaultdict(dict)

for icao, sfinal_df in final_df.groupby("icao24"):
    map_time_traj[icao][sfinal_df.iloc[0]["time"]] = icao + "_1"
    n_traj = 1
    for i in range(1, sfinal_df.shape[0]):
        time = sfinal_df.iloc[i]["time"]
        diff = abs(int(time) - int(sfinal_df.iloc[i]["prev_time"]))
        if diff > TIME_BETWEEN_TRAJS:
            n_traj += 1
        map_time_traj[icao][time] = icao + "_" + str(int(n_traj))

final_df['ref'] = final_df.apply(lambda x: map_time_traj[x.icao24][x.time], axis=1) + '_' + final_df.time.apply(lambda x: pd.to_datetime(x, unit='s').strftime("%d%m%y"))

# Add a distance column and compute cumulative along-track distance for each flight
final_df['dist'] = 0.0
for flight in final_df.ref.unique():
  first = True
  current = final_df[final_df['ref'].isin([flight])] # gets the trajectory of the current flight
  for row in current.itertuples():
    if not(first):
      current_pt = (float(row.lat), float(row.lon))
      delta_dist = haversine(previous_pt, current_pt)
      final_df.loc[row[0],'dist'] = previous_dist + delta_dist
    previous_pt = (float(row.lat), float(row.lon))
    previous_dist = final_df.loc[row[0],'dist']
    first = False


# Load obstacle information and check min height

In [142]:
path_to_obstacles_json = './resources/obstacles.json'

with open(path_to_obstacles_json) as obstacles_database:
    obstacles_data = json.load(obstacles_database)
obs_df = pd.json_normalize(obstacles_data, record_path =['obstacles'])

obs_df['etrs89_x'], obs_df['etrs89_y'] = zip(*obs_df.apply(lambda row: transform_coords(row['lon'], row['lat']), axis=1))
obs_df['gnd_elev'] = obs_df.apply(lambda row: get_elevation(row['etrs89_x'], row['etrs89_y'], dem_src), axis=1)
obs_df = obs_df.drop(columns=['etrs89_x', 'etrs89_y'])

obs_df = obs_df.sort_values(by=['height_m']) # sort obstacles by incresing height, to avoid that the min_hgt profil is wrong if a shorter obstacle comes after a taller one, in case the aircraft is within two obstacles clearance areas


In [152]:
def update_closest_obstacle(final_df, obstacles_df, radius):
    # Iterate over each point in the final_df
    for index, row in final_df.iterrows():
        point = (row['lat'], row['lon'])
        
        # Filter obstacles within the given radius
        obstacles_within_radius = obstacles_df[
            obstacles_df.apply(lambda obs: haversine(point, (obs['lat'], obs['lon'])) <= radius, axis=1)
        ]
        
        # If there are any obstacles within the radius, find the tallest one
        if not obstacles_within_radius.empty:
            tallest_obstacle = obstacles_within_radius.loc[obstacles_within_radius['height_m'].idxmax()]
            final_df.at[index, 'closest_obst_name'] = tallest_obstacle['name']
            final_df.at[index, 'min_hgt'] = GEOID_HEIGHT_M + np.float32(tallest_obstacle['gnd_elev']) + np.float32(tallest_obstacle['height_m']) + ALERT_DELTA_HEIGHT_M

    return final_df

In [153]:
final_df = update_closest_obstacle(final_df, obs_df, 600)

In [155]:
final_df['dip'] = final_df['min_hgt'] - final_df['geoaltitude']

In [156]:
final_df

,time,icao24,lat,lon,velocity,heading,vertrate,callsign,onground,alert,...,prev_time,closest_obst_name,inf_flt,inf_pt,gnd_inf_flt,gnd_inf_pt,min_hgt,ref,dist,dip
0,1719822416,3df741,50.966740,6.948318,51.531781,165.547572,0.00000,HUMMEL1,False,False,...,NaN,ground,False,False,False,False,351.798279,3df741_1_010724,0.000000,31.758279
1,1719822417,3df741,50.966414,6.948471,51.165846,164.845932,-0.97536,HUMMEL1,False,False,...,1.719822e+09,ground,False,False,False,False,351.556396,3df741_1_010724,37.777294,31.516396
2,1719822418,3df741,50.965988,6.948630,50.807694,164.134292,-0.97536,HUMMEL1,False,False,...,1.719822e+09,ground,False,False,False,False,352.370453,3df741_1_010724,86.428655,32.330453
3,1719822419,3df741,50.965530,6.948853,50.115401,162.681062,-0.65024,HUMMEL1,False,False,...,1.719822e+09,ground,False,False,False,False,351.939545,3df741_1_010724,139.747601,31.899545
4,1719822420,3df741,50.965118,6.949075,49.781595,161.939528,-0.65024,HUMMEL1,False,False,...,1.719822e+09,ground,False,False,False,False,351.158844,3df741_1_010724,188.058617,31.118844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3972,1720174101,3d1229,50.912247,7.002300,51.165846,105.154068,-3.57632,DEFCZ,False,False,...,1.720174e+09,ground,False,False,False,False,350.599121,3d1229_1_050724,8261.686918,-243.760879
3973,1720174102,3d1229,50.912134,7.002945,51.165846,105.154068,-3.57632,DEFCZ,False,False,...,1.720174e+09,ground,False,False,False,False,350.719666,3d1229_1_050724,8308.633397,-243.640334
3974,1720174103,3d1229,50.912041,7.003632,51.165846,105.154068,-3.57632,DEFCZ,False,False,...,1.720174e+09,ground,False,False,False,False,350.709930,3d1229_1_050724,8357.874548,-236.030070
3975,1720174104,3d1229,50.911901,7.004318,50.807694,105.865708,-3.90144,DEFCZ,False,False,...,1.720174e+09,ground,False,False,False,False,350.252411,3d1229_1_050724,8408.457912,-236.487589


In [158]:
rhein_coords = [[6.975930879168959,50.95657408794192],
[6.972535124899244,50.95458641663587],
[6.967809452697489,50.95085449055784],
[6.964303258235203,50.94648787705209],
[6.962946874051212,50.94281719236779],
[6.96352513940893,50.93620396292537],
[6.965306071776009,50.93028081346332],
[6.969701292329919,50.91962775943225],
[6.974670336848934,50.91147192727765],
[6.979500669602354,50.90559075596518],
[6.988910832741286,50.89811010115705],
[7.001299970098163,50.89290819036373],
[7.011608685904386,50.89253183470269],
[7.013331310567981,50.89615626376607],
[7.001160198607601,50.89750937089659],
[6.991533142700823,50.90103254802452],
[6.988667339298418,50.90270433843045],
[6.983479655838492,50.90851708398598],
[6.975279465952471,50.91906362354751],
[6.970153101800916,50.93012572612207],
[6.970658769054046,50.93156686745481],
[6.968078026948048,50.93770005507804],
[6.968636570741024,50.94353672207991],
[6.972110561853963,50.94977637503464],
[6.978430041779998,50.95436079405016],
[6.984223870632302,50.9521389418798],
[6.994914076799508,50.95996315483612],
[6.997507875246713,50.96355172063916],
[6.993574311759938,50.96491142024293],
[6.975930879168959,50.95657408794192]]

rhein_polygon = Polygon(rhein_coords)

points = gpd.GeoSeries([Point(xy) for xy in zip(final_df['lon'], final_df['lat'])])

final_df['in_rhein'] = points.within(rhein_polygon)


In [171]:
final_df[(final_df['dip'] > 0) & ~(final_df['in_rhein']) & (final_df['callsign'] == 'N31VX   ')]

,time,icao24,lat,lon,velocity,heading,vertrate,callsign,onground,alert,...,closest_obst_name,inf_flt,inf_pt,gnd_inf_flt,gnd_inf_pt,min_hgt,ref,dist,dip,in_rhein
3485,1720102847,a34669,50.963837,6.934303,78.501188,151.419507,0.32512,N31VX,False,False,...,ground,False,False,False,False,350.753235,a34669_1_040724,398.017916,0.233235,False
3503,1720102865,a34669,50.953054,6.943512,75.003042,153.083445,0.32512,N31VX,False,False,...,Koelnturm (Mediapark),False,False,False,False,570.595825,a34669_1_040724,1760.127494,212.455825,False
3504,1720102866,a34669,50.952347,6.944027,75.004806,153.962196,0.00000,N31VX,False,False,...,Koelnturm (Mediapark),False,False,False,False,570.595825,a34669_1_040724,1846.620512,212.455825,False
3505,1720102867,a34669,50.951752,6.944473,74.771564,153.434949,-0.32512,N31VX,False,False,...,Koelnturm (Mediapark),False,False,False,False,570.595825,a34669_1_040724,1919.778205,212.455825,False
3506,1720102868,a34669,50.951065,6.945067,74.780412,152.553545,0.00000,N31VX,False,False,...,Koelnturm (Mediapark),False,False,False,False,570.595825,a34669_1_040724,2006.727045,212.455825,False
3507,1720102869,a34669,50.950493,6.945496,75.018919,152.204860,0.32512,N31VX,False,False,...,Koelnturm (Mediapark),False,False,False,False,570.595825,a34669_1_040724,2077.051293,212.455825,False
3508,1720102870,a34669,50.949966,6.945957,74.324251,152.370753,0.32512,N31VX,False,False,...,Koelnturm (Mediapark),False,False,False,False,570.595825,a34669_1_040724,2143.972600,212.455825,False
3509,1720102871,a34669,50.949371,6.946477,74.324251,152.370753,0.65024,N31VX,False,False,...,Koelnturm (Mediapark),False,False,False,False,570.595825,a34669_1_040724,2219.495392,212.455825,False
3510,1720102872,a34669,50.948864,6.946869,74.324251,152.370753,0.97536,N31VX,False,False,...,Koelnturm (Mediapark),False,False,False,False,570.595825,a34669_1_040724,2282.239009,204.835825,False
3511,1720102873,a34669,50.948259,6.947403,73.868856,152.185706,1.30048,N31VX,False,False,...,Koelnturm (Mediapark),False,False,False,False,570.595825,a34669_1_040724,2359.233044,204.835825,False
